In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

In [4]:
import ray
import pandas as pd
from utils import helper, config, rayer
from ml.models.ensemble_v2 import Ensemble
from ml.xai.model.explainable import Explainable


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from ml.models import common


In [5]:
rayer.get_global_cluster(num_cpus=5)
!ray status --address='raycluster-autoscaler-head-svc.dev.svc.cluster.local:6379'

======== Autoscaler status: 2022-11-23 06:52:49.788828 ========
Node status
---------------------------------------------------------------
Healthy:
 1 head-group
 2 small-group
Pending:
 (no pending nodes)
Recent failures:
 (no failures)

Resources
---------------------------------------------------------------
Usage:
 0.0/5.0 CPU
 0.00/16.764 GiB memory
 0.00/4.909 GiB object_store_memory

Demands:
 {'CPU': 1}: 5+ from request_resources()


In [3]:
!ray status --address='raycluster-autoscaler-head-svc.dev.svc.cluster.local:6379'

======== Autoscaler status: 2022-11-17 08:37:13.186794 ========
Node status
---------------------------------------------------------------
Healthy:
 1 head-group
 3 small-group
Pending:
 (no pending nodes)
Recent failures:
 (no failures)

Resources
---------------------------------------------------------------
Usage:
 2.0/7.0 CPU
 0.00/24.214 GiB memory
 0.00/7.114 GiB object_store_memory

Demands:
 {'CPU': 1}: 5+ from request_resources()


In [7]:
# ray.autoscaler.sdk.request_resources(num_cpus=0)

In [6]:
df_X, df_y = helper.get_covid_dataset()
df_X = df_X.drop(['location'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.33, random_state=config.rand_state)

In [7]:
ss = StandardScaler()

X_train_scalar = pd.DataFrame(ss.fit_transform(X_train.copy()), columns=X_train.columns)
X_test_scalar = pd.DataFrame(ss.fit_transform(X_test.copy()), columns=X_test.columns)


In [8]:
list_base_models = ['BriskXGBoost', 'SlugXGBoost', 'SlugANN', 'SlugRF', 'SlugKNN', 'BriskBagging']

ensemble_set = Ensemble(
                        list_base_models = list_base_models,
                        n_trials=10, ### for all models
                        boosted_round=50, ### for tree models only
                        epochs=15, ### for DNN models

                        ensemble_boosted_round=10, ### for ensemble model which is also a free model
                        ensemble_n_trials=3,
                                )



In [9]:
ensemble_set.fetch_models(X_train_scalar, X_test_scalar, y_train, y_test, threshold=None)

2022-11-23T06:52:56CST : INFO : ensemble_v2 : fetch_models : 108 : Message : Ensemble: starting discovery process for models [<ml.models.base.v2.brisk_xgboost.BriskXGBoost object at 0x7f7d580b5940>, <ml.models.base.v2.slug_xgboost.SlugXGBoost object at 0x7f7d580b4730>, <ml.models.base.v2.slug_ann.SlugANN object at 0x7f7d580b4910>, <ml.models.base.v2.slug_rf.SlugRF object at 0x7f7d580b4880>, <ml.models.base.v2.slug_knn.SlugKNN object at 0x7f7d580b4520>, <ml.models.base.v2.brisk_bagging.BriskBagging object at 0x7f7d580b4670>]
(raylet) [2022-11-23 04:53:00,387 E 74 74] (raylet) worker_pool.cc:1108: Failed to send exit request: GrpcUnavailable: RPC Error message: Connection reset by peer; RPC Error details: 
(__run_discoveries__ pid=581, ip=10.10.48.229) 2022-11-23T04:53:00PST : INFO : brisk_xgboost : __discover_model__ : 109 : Message : brisk_xgb: Starting training for trials:10, boosted rounds: 50, max depth: 10
(__run_discoveries__ pid=581, ip=10.10.48.229) 2022-11-23T04:53:01PST : INFO

In [12]:
ensemble_set.base_model_scores

[[-0.780242599582226,
  -0.8618234540916516,
 [-0.36896820977009215,
  -0.6666532672303094,
 [0.5712783142765323,
  0.16147673240819727,
 [0.6471896784078507,
  0.4251536230476787,
 [0.6695206594539054,
  0.5401046443169623,
 [0.9506921261413173,
  0.5390826970012106,

In [13]:
attr_algos = ['IG', 'SHAP', 'GradientSHAP']

In [14]:
ex = Explainable(ensemble_set, df_X)
ex.get_attr(attr_algos)

2022-11-23T06:55:12CST : INFO : explainable : get_attr : 148 : Message : attribution methods  ['ig', 'shap', 'gradientshap']
2022-11-23T06:55:12CST : INFO : explainable : get_attr : 151 : Message : calculating variable importance on  brisk_xgb
2022-11-23T06:55:13CST : INFO : explainable : get_attr : 151 : Message : calculating variable importance on  slug_xgb
2022-11-23T06:55:13CST : INFO : explainable : get_attr : 151 : Message : calculating variable importance on  slug_ann
2022-11-23T06:55:13CST : INFO : explainable : get_attr : 151 : Message : calculating variable importance on  slug_rf
2022-11-23T06:55:13CST : INFO : explainable : get_attr : 151 : Message : calculating variable importance on  slug_knn
2022-11-23T06:55:13CST : INFO : explainable : get_attr : 151 : Message : calculating variable importance on  brisk_bagging
(__get_ig_attr__ pid=581, ip=10.10.48.229) IPython could not be loaded!
(__get_gs_attr__ pid=526, ip=10.10.105.191) /tmp/ray/session_2022-11-23_00-55-48_777969_8/

,brisk_xgb_shap,slug_xgb_shap,slug_ann_ig,slug_ann_shap,slug_ann_gradientshap,slug_rf_shap,slug_knn_shap,brisk_bagging_shap
new_cases_per_million,0.000000,8.579548e-02,3.759912e+06,1.884617e+06,3.051007e+06,763.946640,14.309589,0.530441
population_cov,0.000273,5.284828e-01,3.619563e+08,3.937101e+08,5.367331e+08,5.718554,39.301761,0.000000
life_expectancy_cov,0.000000,0.000000e+00,4.200456e+02,3.072453e+01,4.508412e+01,1.469238,0.612097,0.000000
Population,0.000000,1.901932e-01,4.727798e+08,5.155930e+08,7.027543e+08,12.663485,38.241803,0.000000
Area_km2,0.000253,1.583993e-03,3.285954e+06,3.684905e+06,5.693634e+06,1.066764,28.207985,0.000000
Density_km2,0.000000,1.061057e-03,2.110041e+04,8.469975e+03,6.852935e+03,3.000385,0.478778,0.217915
Year_x,0.000000,0.000000e+00,1.499756e+03,0.000000e+00,3.092911e-02,0.000000,0.000000,0.000000
Meningitis,0.000000,0.000000e+00,3.777460e-02,4.144062e-02,7.853202e-01,1.300893,0.412961,0.000000
Neoplasms,0.000000,0.000000e+00,8.472476e-01,1.947015e-01,3.114214e-01,18.083464,0.255938,0.041545
"Fire, heat, and hot substances",0.000000,1.031880e-03,1.955676e-02,8.841133e-03,5.811997e-01,0.348636,0.304805,0.000000


(__get_shapley_kernel_attr__ pid=526, ip=10.10.105.191) 2022-11-23T04:55:26PST : INFO : _kernel : solve : 549 : Message : np.sum(w_aug) = 39.000000000000014
(__get_shapley_kernel_attr__ pid=526, ip=10.10.105.191) 2022-11-23T04:55:26PST : INFO : _kernel : solve : 550 : Message : np.sum(self.kernelWeights) = 1.0000000000000004
(__get_shapley_kernel_attr__ pid=526, ip=10.10.105.191) The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
(__get_shapley_kernel_attr__ pid=526, ip=10.10.105.191) If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:
(__get_shapley_kernel_attr__ pid=526, ip=10.10.105.191) 
(__get_shapley_kernel_attr__ pid=526, ip=10.10.105.191) from sklearn.pipeline import make_pipeline
(__get_shapley_kernel_attr__ pid=526, ip=10.10.105.191) 
(__get_shapley_kernel_attr__ pid=526, ip=10.10.105.191) model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())


In [15]:
ex.df_scores

,brisk_xgb_shap,slug_xgb_shap,slug_ann_ig,slug_ann_shap,slug_ann_gradientshap,slug_rf_shap,slug_knn_shap,brisk_bagging_shap
new_cases_per_million,0.000000,8.579548e-02,3.759912e+06,1.884617e+06,3.051007e+06,763.946640,14.309589,0.530441
population_cov,0.000273,5.284828e-01,3.619563e+08,3.937101e+08,5.367331e+08,5.718554,39.301761,0.000000
life_expectancy_cov,0.000000,0.000000e+00,4.200456e+02,3.072453e+01,4.508412e+01,1.469238,0.612097,0.000000
Population,0.000000,1.901932e-01,4.727798e+08,5.155930e+08,7.027543e+08,12.663485,38.241803,0.000000
Area_km2,0.000253,1.583993e-03,3.285954e+06,3.684905e+06,5.693634e+06,1.066764,28.207985,0.000000
Density_km2,0.000000,1.061057e-03,2.110041e+04,8.469975e+03,6.852935e+03,3.000385,0.478778,0.217915
Year_x,0.000000,0.000000e+00,1.499756e+03,0.000000e+00,3.092911e-02,0.000000,0.000000,0.000000
Meningitis,0.000000,0.000000e+00,3.777460e-02,4.144062e-02,7.853202e-01,1.300893,0.412961,0.000000
Neoplasms,0.000000,0.000000e+00,8.472476e-01,1.947015e-01,3.114214e-01,18.083464,0.255938,0.041545
"Fire, heat, and hot substances",0.000000,1.031880e-03,1.955676e-02,8.841133e-03,5.811997e-01,0.348636,0.304805,0.000000


In [24]:
ex.df_scores['cols'] = ex.df_scores.index
ex.df_scores.to_csv('df_scores_3.csv', sep=';', index=False)

In [5]:
df = pd.read_csv('df_scores_3.csv', sep=';')

In [6]:
df_scores_ranked = pd.DataFrame()

for col in df:
    if col != 'cols':
        df_scores_ranked[col] = df[col].rank(na_option = 'bottom', ascending=True, method='max', pct=False)
        # df_scores_ranked
        df_scores_ranked.replace(to_replace = df_scores_ranked.min(), value = 0, inplace=True)


In [7]:
#df_scores_ranked = df_scores_ranked.mode(axis=1)[0]

In [8]:
df_scores_ranked['cols'] = df['cols']

In [9]:
df_scores_ranked

,brisk_xgb_shap,slug_xgb_shap,slug_ann_ig,slug_ann_shap,slug_ann_gradientshap,slug_rf_shap,slug_knn_shap,brisk_bagging_shap,cols
0,0.0,37.0,38.0,37.0,37.0,40.0,37.0,38.0,new_cases_per_million
1,40.0,40.0,39.0,39.0,39.0,28.0,40.0,0.0,population_cov
2,0.0,0.0,34.0,34.0,34.0,17.0,33.0,0.0,life_expectancy_cov
3,0.0,39.0,40.0,40.0,40.0,31.0,39.0,0.0,Population
4,39.0,29.0,37.0,38.0,38.0,13.0,38.0,0.0,Area_km2
5,0.0,27.0,36.0,36.0,36.0,22.0,26.0,37.0,Density_km2
6,0.0,0.0,35.0,0.0,0.0,0.0,0.0,0.0,Year_x
7,0.0,0.0,12.0,17.0,27.0,15.0,23.0,0.0,Meningitis
8,0.0,0.0,30.0,30.0,11.0,35.0,12.0,35.0,Neoplasms
9,0.0,26.0,9.0,8.0,19.0,7.0,14.0,0.0,"Fire, heat, and hot substances"


In [14]:
res = df_scores_ranked.mode(axis=1, numeric_only=True).mean(axis=1)
res.index = df_scores_ranked['cols']

In [17]:
res.sort_values(ascending=False)

cols
Population                                    40.000000
population_cov                                39.500000
Area_km2                                      38.000000
new_cases_per_million                         37.000000
Density_km2                                   36.000000
Total                                         22.375000
Neoplasms                                     21.666667
Cirrhosis and other chronic liver diseases    21.125000
Lower respiratory infections                  20.500000
life_expectancy_cov                           17.000000
Self-harm                                     13.666667
Neonatal disorders                            12.500000
Road injuries                                 10.000000
Drowning                                       5.500000
Nutritional deficiencies                       2.500000
Diabetes mellitus                              0.000000
Poisonings                                     0.000000
Protein-energy malnutrition                